In [109]:
import fiona
#from fiona.crs import to_string
#import pprint
#import shapely
#from shapely.geometry import Polygon
import geopandas as gpd
import glob
from rasterstats import zonal_stats
import pandas as pd

In [110]:
shp = 'data/geo/countries/countries_nf2.shp'
gdf = gpd.GeoDataFrame.from_file(shp); gdf.head(3)

,ADMIN,CONTINENT,ISO_A3,REGION_UN,REGION_WB,SUBREGION,WB_A3,geometry
0,Aruba,North America,ABW,Americas,Latin America & Caribbean,Caribbean,ABW,POLYGON ((-69.99693762899992 12.57758209800004...
1,Afghanistan,Asia,AFG,Asia,South Asia,Southern Asia,AFG,"POLYGON ((71.04980228700009 38.40866445000009,..."
2,Angola,Africa,AGO,Africa,Sub-Saharan Africa,Middle Africa,AGO,(POLYGON ((11.73751945100014 -16.6925779829998...


In [111]:
def get_tif_ids(tifs_dir):
    glob_str = tifs_dir + '/*.tif'
    li_img = glob.glob(glob_str)
    li_ids = []
    for i in li_img:
        first_chunk = i.split('/')[3]
        second_chunk = first_chunk.split('.')[0]
        li_ids.append(second_chunk)
    return li_ids
get_tif_ids('data/geo/images')

['F101992',
 'F101993',
 'F101994',
 'F121994',
 'F121995',
 'F121996',
 'F121997',
 'F121998',
 'F121999',
 'F141997',
 'F141998',
 'F141999',
 'F142000',
 'F142001',
 'F142002',
 'F142003',
 'F152000',
 'F152001',
 'F152002',
 'F152003',
 'F152004',
 'F152005',
 'F152006',
 'F152007',
 'F162004',
 'F162005',
 'F162006',
 'F162007',
 'F162008',
 'F162009',
 'F182010',
 'F182011',
 'F182012',
 'F182013']

In [112]:
raster = 'data/geo/images/F142002.v4b_web.stable_lights.avg_vis.tif'
shp = 'data/geo/countries/countries_nf2.shp'
stats = zonal_stats(shp, raster)
stats[1].keys()

['count', 'max', 'mean', 'min']

In [74]:
li_stats = []
for i in range(len(stats)):
    li_stats.append(stats[i]['mean'])
li_stats[:3]

[34.333333333333336, 0.03426151517975618, 0.03442736150843421]

In [78]:
s_stats = pd.Series(li_stats, name='light'); s_stats.head(3)

0    34.333333
1     0.034262
2     0.034427
Name: light, dtype: float64

In [82]:
gdf = gdf.join(s_stats); gdf.head(3)

,ADMIN,CONTINENT,ISO_A3,REGION_UN,REGION_WB,SUBREGION,WB_A3,geometry,light
0,Aruba,North America,ABW,Americas,Latin America & Caribbean,Caribbean,ABW,POLYGON ((-69.99693762899992 12.57758209800004...,34.333333
1,Afghanistan,Asia,AFG,Asia,South Asia,Southern Asia,AFG,"POLYGON ((71.04980228700009 38.40866445000009,...",0.034262
2,Angola,Africa,AGO,Africa,Sub-Saharan Africa,Middle Africa,AGO,(POLYGON ((11.73751945100014 -16.6925779829998...,0.034427


In [104]:
# need to create a for loop that finds zonal stats for each country for each year
def zonal_to_shp(tifs_dir, shp_path):
    tifs_glob_str = tifs_dir + '/*.tif'
    tifs = glob.glob(tifs_glob_str)
    ids = get_tif_ids(tifs_dir)
    geodataframe = gpd.GeoDataFrame.from_file(shp_path)
    for idx, tif in enumerate(tifs):
        stats = zonal_stats(shp_path, tif)
        li_stats = []
        for i in range(len(stats)):
            li_stats.append(stats[i]['mean'])
        s_stats = pd.Series(li_stats, name=ids[idx])
        geodataframe = geodataframe.join(s_stats)
    return geodataframe

In [113]:
zonal_stats('data/geo/countries/countries_nf2.shp', 'data/geo/images/F142002.v4b_web.stable_lights.avg_vis.tif')

[{'count': 201, 'max': 63.0, 'mean': 34.333333333333336, 'min': 3.0},
 {'count': 901332, 'max': 60.0, 'mean': 0.03426151517975618, 'min': 0.0},
 {'count': 1490181, 'max': 63.0, 'mean': 0.03442736150843421, 'min': 0.0},
 {'count': 100, 'max': 40.0, 'mean': 16.65, 'min': 0.0},
 {'count': 43766, 'max': 61.0, 'mean': 1.6822647717406205, 'min': 0.0},
 {'count': 2190, 'max': 0.0, 'mean': 0.0, 'min': 0.0},
 {'count': 711, 'max': 62.0, 'mean': 21.47960618846695, 'min': 0.0},
 {'count': 80358, 'max': 63.0, 'mean': 8.934642474924711, 'min': 0.0},
 {'count': 3923652, 'max': 63.0, 'mean': 0.5758780340356382, 'min': 0.0},
 {'count': 45117, 'max': 62.0, 'mean': 1.2807145865194938, 'min': 0.0},
 {'count': 216, 'max': 35.0, 'mean': 10.787037037037036, 'min': 0.0},
 {'count': 87135532, 'max': 11.0, 'mean': 4.108542081317642e-06, 'min': 0.0},
 {'count': 2, 'max': 0.0, 'mean': 0.0, 'min': 0.0},
 {'count': 12908, 'max': 5.0, 'mean': 0.34575457080880073, 'min': 0.0},
 {'count': 552, 'max': 62.0, 'mean': 11

In [114]:
gdf = zonal_to_shp('data/geo/images', 'data/geo/countries/countries_nf2.shp')

In [141]:
gdf.head(3)

,ADMIN,CONTINENT,ISO_A3,REGION_UN,REGION_WB,SUBREGION,WB_A3,geometry,F101992,F101993,...,F162004,F162005,F162006,F162007,F162008,F162009,F182010,F182011,F182012,F182013
0,Aruba,North America,ABW,Americas,Latin America & Caribbean,Caribbean,ABW,POLYGON ((-69.99693762899992 12.57758209800004...,30.731343,31.004975,...,36.492537,36.029851,35.741294,37.577114,36.228856,33.386139,35.336634,36.316832,37.621891,36.671642
1,Afghanistan,Asia,AFG,Asia,South Asia,Southern Asia,AFG,"POLYGON ((71.04980228700009 38.40866445000009,...",0.027943,0.020531,...,0.083359,0.095992,0.109460,0.124478,0.098875,0.107979,0.196381,0.177007,0.212513,0.194248
2,Angola,Africa,AGO,Africa,Sub-Saharan Africa,Middle Africa,AGO,(POLYGON ((11.73751945100014 -16.6925779829998...,0.028212,0.028248,...,0.049086,0.041765,0.057345,0.069067,0.086209,0.100469,0.165455,0.151740,0.179941,0.181871


In [116]:
import pickle

In [129]:
with open('data/geo/pickles/zonal_stats_c.pickle', 'wb') as f:
    pickle.dump(gdf, f)

In [146]:
wb = pd.read_csv('data/econ/wb.csv')
wb_const = wb[wb['Series Name'] == 'GDP at market prices (constant 2005 US$)']; wb_const.shape

(248, 28)

In [145]:
# drop nans
wb = wb_const[wb_const.notnull()]

,Country Name,Country Code,Series Name,Series Code,1992,1993,1994,1995,1996,1997,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Afghanistan,AFG,GDP at market prices (constant 2005 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,6.623602e+09,7.533699e+09,7.805769e+09,9.446592e+09,1.024325e+10,1.086949e+10,1.243847e+10,1.268216e+10,1.284862e+10,NaN
4,Albania,ALB,GDP at market prices (constant 2005 US$),NY.GDP.MKTP.KD,3.553156e+09,3.894259e+09,4.217482e+09,4.778408e+09,5.213243e+09,4.648239e+09,...,8.601641e+09,9.109137e+09,9.795055e+09,1.012319e+10,1.049876e+10,1.076648e+10,1.091936e+10,1.104057e+10,1.128015e+10,NaN
8,Algeria,DZA,GDP at market prices (constant 2005 US$),NY.GDP.MKTP.KD,6.715227e+10,6.574207e+10,6.515039e+10,6.762611e+10,7.039878e+10,7.117316e+10,...,1.049530e+11,1.085210e+11,1.106910e+11,1.124620e+11,1.165110e+11,1.197740e+11,1.237260e+11,1.271900e+11,1.320240e+11,NaN
12,American Samoa,ASM,GDP at market prices (constant 2005 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,Andorra,ADO,GDP at market prices (constant 2005 US$),NY.GDP.MKTP.KD,1.706472e+09,1.688870e+09,1.729119e+09,1.776799e+09,1.859416e+09,2.028021e+09,...,3.397072e+09,3.399723e+09,3.107542e+09,2.988896e+09,2.829051e+09,2.693181e+09,2.645780e+09,2.644100e+09,NaN,NaN
20,Angola,AGO,GDP at market prices (constant 2005 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,Antigua and Barbuda,ATG,GDP at market prices (constant 2005 US$),NY.GDP.MKTP.KD,6.164163e+08,6.496418e+08,6.907487e+08,6.618974e+08,7.065063e+08,7.410251e+08,...,1.130787e+09,1.238200e+09,1.239080e+09,1.089944e+09,1.012090e+09,9.939385e+08,1.033890e+09,1.030310e+09,1.079992e+09,NaN
28,Arab World,ARB,GDP at market prices (constant 2005 US$),NY.GDP.MKTP.KD,6.862830e+11,7.110850e+11,7.362090e+11,7.563530e+11,7.926110e+11,8.263010e+11,...,1.265030e+12,1.337620e+12,1.422390e+12,1.443520e+12,1.511350e+12,1.563780e+12,1.667150e+12,1.714330e+12,1.753350e+12,NaN
32,Argentina,ARG,GDP at market prices (constant 2005 US$),NY.GDP.MKTP.KD,1.617710e+11,1.713270e+11,1.813260e+11,1.761670e+11,1.859030e+11,2.009820e+11,...,2.393680e+11,2.584360e+11,2.663820e+11,2.665160e+11,2.917050e+11,3.161690e+11,3.187040e+11,3.279000e+11,3.293870e+11,NaN
36,Armenia,ARM,GDP at market prices (constant 2005 US$),NY.GDP.MKTP.KD,2.088161e+09,1.904402e+09,2.007240e+09,2.145740e+09,2.271596e+09,2.347038e+09,...,5.547234e+09,6.309934e+09,6.745319e+09,5.790858e+09,5.918256e+09,6.196414e+09,6.642556e+09,6.861761e+09,7.101922e+09,NaN
